# Importing Libraries

In [1]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
import os
import smtplib, ssl
import config
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email import encoders
import pywhatkit
from datetime import *
import boto3
import time

# Creating Live Dataset 

In [2]:
client=boto3.client('ec2')



# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions and this function return the cropped images
def create_dataset(img):
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found 
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if create_dataset(frame) is not None:
        count += 1
        face = cv2.resize(create_dataset(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './dataset/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) & 0xFF==ord('q') or count == 150: # Press q to quit 
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

# Training the Model

In [3]:
# Get the training data we previously made
data_path = './dataset/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)
my_model  = cv2.face_LBPHFaceRecognizer.create()

# Let's train our model 
my_model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")

Model trained sucessefully


# Capturing photo of the user

In [4]:
#Capturing individual photo to send to the allocated email id.
cap = cv2.VideoCapture(0)
ret , photo = cap.read()

cv2.imwrite("pic1.jpg", photo)
cap.release()

print("Opening Camera to verify the face...")

Opening Camera to verify the face...


# Face recognition through Model

In [5]:
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    image, face = face_detector(frame)
    
    
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results = my_model.predict(face)
        # harry_model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        #os.system('sleep 15')
        if confidence > 90:
            
            cv2.putText(image, "Hey Utkarsh", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
            
            #os.system("chrome https://www.google.com/search?q=vimal+daga")
            #os.system("wmplayer   c:\lw.mp3")
            #os.system('sleep 10')
            
            sender = os.environ.get('SENDER_EMAIL')
            receivers = [os.environ.get('RECIEVER_EMAIL')]
            body_of_email = 'Hello, the following person tried to use your device'
            
            message = MIMEMultipart()
            message['Subject'] = 'Face_detected'
            message['From'] = sender
            message['To'] = ','.join(receivers)

            #Adds a csv file as an attachment to the email 
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(open('pic1.jpg', 'rb').read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', 'attachment; filename ="face.jpg"')
            message.attach(part)
            
            
            smtpObj = smtplib.SMTP(host='smtp.gmail.com', port=587)
            smtpObj.starttls()
            smtpObj.login(sender, os.environ.get('E_PASS'))
            #message = """Stage 1 completed!! """
            smtpObj.sendmail(sender, receivers, message.as_string())
            smtpObj.quit()
            print("Mail sent successfully")
            
            
            pywhatkit.sendwhatmsg(os.environ.get('WHATSAPP_USER_1'), "Launching AWS instance with 5GB EBS volume ",datetime.now().hour,datetime.now().minute+1)
            print("Whatsapp user notified")
            
            keyname='keyhadoopNN'
            instanceName="TESTING_INSTANCE"
            
            ebsName="TESTING_EBS"
            
            instances=client.run_instances(
                ImageId='ami-0e306788ff2473ccb',
                MinCount=1,
                MaxCount=1,
                InstanceType='t2.micro',
                KeyName='{}'.format(keyname),
                Placement={
                    'AvailabilityZone': 'ap-south-1a'
                          },
                TagSpecifications=[
                    {
                        'ResourceType': 'instance',
                        'Tags': [
                            {
                                'Key': 'Name',
                                'Value': '{}'.format(instanceName)
                            },
                        ]
                    },
                ],
            )
            print("AWS instance launched")
            instance_id=instances['Instances'][0]['InstanceId']
            volumes=client.create_volume(
                AvailabilityZone='ap-south-1a',
                Size=5,
                VolumeType='gp2',
                TagSpecifications=[
                    {
                        'ResourceType': 'volume',
                        'Tags': [
                            {
                                'Key': 'Name',
                                'Value': '{}'.format(ebsName)
                            },
                        ]
                    },
                ],
            )
            print("EBS Volume launched")
            volume_id=volumes['VolumeId']
            print("Getting instance ready in 60 sec to attach the volume")
            #os.system('powershell Start-Sleep -m 60000')
            time.sleep(60)
            response = client.attach_volume(
                Device='/dev/sdh',
                InstanceId='{}'.format(instance_id),
                VolumeId='{}'.format(volume_id),
            )
            
            print("AWS instance launched along with the attached volume ")
            
            break
            
        elif cv2.waitKey(1)==13:
            break
         
        else:
            
            cv2.putText(image, "Face not recognised", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()

Mail sent successfully
In 35 seconds web.whatsapp.com will open and after 20 seconds message will be delivered
Whatsapp user notified
AWS instance launched
EBS Volume launched
Getting instance ready in 60 sec to attach the volume
AWS instance launched along with the attached volume 
